In [1]:
import sys
import re
from os import walk
from pathlib import Path
from functools import reduce

In [3]:
ROOT_PATH = "/Users/janschill/code/rails/rails"

In [4]:
def full_path(file_name):
    return ROOT_PATH + file_name

In [5]:
# Number of lines
def LOC(file):
    return sum([1 for line in open(file)])

In [97]:
def extract_from_line(name, line):
    if re.search("^([\s]*#)", line): # ignore comments
        return None
    elif name == 'module' or 'class':
        namespace = re.search("^([\s]*(module|class) (\S+))", line)
        namespace = str(namespace.group(1))
        return namespace.replace('module', '').replace(' ', '')
    else:
        x = re.search("" + name + " (\S+)", line)
        return None if x == None else str(x.group(1))

In [98]:
extract_from_line('module', 'module ActionCable')

'ActionCable'

In [22]:
# Number of methods
def number_of(file, key_word):
    return sum([1 for line in open(file) if extract_from_line(key_word, line)])

def extract_requires(file):
    requires = []
    for line in open(file):
        require = extract_from_line('require', line)
        if require:
            requires.append(require)
    return requires

In [8]:
def number_of_modules(file):
    return number_of(file, 'module')

def number_of_methods(file):
    return number_of(file, 'def')

def number_of_requires(file):
    return number_of(file, 'require')

In [9]:
directories = []
for (dirpath, dirnames, filenames) in walk(ROOT_PATH):
    directories.extend(dirnames)
    break
directories = list(filter(lambda x: x[0] != ".", directories)) # don't use hidden dirs

In [38]:
def get_files(path, file_extension):
    files = Path(path).rglob("*." + file_extension)
    meta_data_files = []
    for file in files:
        meta_data_files.append({
            'filename': str(file),
            'LOC': LOC(file),
            'no_modules': number_of_modules(file),
            'no_methods': number_of_methods(file),
            'no_requires': len(extract_requires(file)),
            'requires': extract_requires(file)
        })
    return meta_data_files

In [11]:
def reduce_by_key(list, key):
    sum = 0
    for el in list:
        sum = sum + el[key]
    return sum

In [31]:
rails_modules = {}
for directory in directories:
    files = get_files(full_path('/' + directory + '/'), 'rb')
    average_LOC = int(reduce_by_key(files, 'LOC') / len(files))
    average_NOM = int(reduce_by_key(files, 'no_methods') / len(files))
    average_requires = int(reduce_by_key(files, 'no_requires') / len(files))
    rails_modules[directory] = {
        'files': files,
        'average LOC': average_LOC ,
        'average NOM': average_NOM,
        'average requires': average_requires }
rails_modules = {k: v for k, v in sorted(rails_modules.items(), key = lambda item: item[0])}

In [35]:
from tabulate import tabulate
format = "{:<15}{:<1}{:<10}{:<1}{:<7}{:<1}{:<10}{:<1}{:<10}"
print(format.format("Module", " | ", "Ruby files", " | ", "Ø LOC", " | ", "Ø Methods", " | ", "Ø Requires"))
print('----------------+------------+---------+------------+-----------')
for k, v in rails_modules.items():
    print(format.format(k, ' | ' , len(v['files']), ' | ', v['average LOC'], ' | ', v['average NOM'], ' | ', v['average requires']))

Module          | Ruby files | Ø LOC   | Ø Methods  | Ø Requires
----------------+------------+---------+------------+-----------
actioncable     | 86         | 80      | 6          | 1         
actionmailbox   | 93         | 34      | 1          | 0         
actionmailer    | 41         | 140     | 9          | 1         
actionpack      | 325        | 202     | 16         | 1         
actiontext      | 74         | 35      | 2          | 0         
actionview      | 186        | 231     | 18         | 1         
activejob       | 118        | 70      | 5          | 1         
activemodel     | 130        | 118     | 8          | 1         
activerecord    | 853        | 160     | 12         | 1         
activestorage   | 124        | 62      | 3          | 1         
activesupport   | 469        | 119     | 10         | 1         
ci              | 1          | 22      | 0          | 2         
guides          | 22         | 69      | 3          | 3         
railties        | 278    

In [37]:
# ActionCable
action_cable = rails_modules['actioncable']
